# Basics of Radiometric Correction for Hyperspectral Data
> Summary

- toc: true 
- badges: true
- comments: true
- categories: []
- image: https://i.insider.com/59356950b74af41b008b64fa
- hide: true
- search_exclude: true
- permalink: /:year/:month/:day/:title

# Introduction
The atmosphere is all nice and necessary for daily life, but it sure does make it tricky to obtain picture perfect hyperspectral images taken by satellites. While we shake our fist at refraction, scattering, and absorption corrupting our data, we are simultaneously amazed at the blue sky produced by the same effects and make it a common theme in desktop wallpapers. We can't blame all our inadequacies on atmospheric effects though. Hyperspectral images also contain sensor errors such as striping, and smile and keystone artefacts which need correcting. These correction procedures require models as we shall see towards the end of this blog post.

There will be some referencecs to Hyperion data so let's understand that. 

## Hyperion Data
Hyperion is the first satellite hyperspectral imager on the Earth Observing-1 [(EO-1 Satellite)](http://eo1.gsfc.nasa.gov). This satellite was part of NASA's New Millennium Program Earth Observing series and sat in a sun-synchronous orbit{% fn 1 %}. The hyperspectral sensor had 220 spectral bands and a spatial resolution of 30 m. The images are taken via pushbroom which means stripes (with 7.5 km swath width in the case of Hyperion) are taken sequentially to build a 2D image with 220 channels. 

Hyperion data is taken above the atmosphere so it will be affected by the atmosphere. 

# Atmospheric Effects
The atmosphere contains molecules such as water, carbon dioxide, ozone, and aerosols. These impede the path that light takes from the scene towards the imaging sensor. The major effects of the atmosphere are: refraction, scattering, and absorption.


## Refraction
Refraction is the bending of light as it enters a medium with a different refractive index. It is refraction that makes a straw appear bent in a glass of water. Every medium has a refractive index $n$ defined as
$$ n = c/v $$
where $c$ is the speed of light in vacuum and $v$ is the speed of light in the medium. 

> Note: When $n$ also depends on the wavelength $\lambda$, we also get dispersion. Refractive index is also defined as $n =\sqrt{\epsilon_r}$ where $\epsilon_r$ is the relative permittivity of the medium. 

We can predict the refracted light angle $\theta_2$ knowing the refractive indicies of two mediums $n_1,n_2$ and incident angle $\theta_1$ by Snell's law
$$ n_1 \sin\theta_1 = n_2 \sin\theta_2 .$$
This procedure can be repeated as light moves into denser (and higher refractive index) regions of the atmosphere as shown in Figure 1.


![](../images/2020-04-27-basics_of_radiometric_correction_files/refraction.png "Figure 1: Light path through atmosphere with refraction effects exaggerated{% fn 2 %}."){:width="80%"}


## Scattering
Atmospheric scatter is caused by many diffuse reflections of light off particles in the air. There are three regimes for scattering: Rayleigh, Mie, and optical depending on the ratio between the incident light wavelength and particle size. Rayleigh when the wavelength $\lambda$ is much smaller than the particle size $d$, optical when the wavelength is much larger than the partical size, and Mie inbetween Rayleigh and optical where there are resonances $0.1 < \lambda/d < 10$. This is aptly described in Figure 2.

![](../images/2020-04-27-basics_of_radiometric_correction_files/scattering.png "Figure 2: Scattering regimes{% fn 3 %}."){:width="80%"}


Assuming Rayleigh scattering, the intensity $I$ of light scattered is given by
$$ I = I_0\frac{1+\cos^2\theta}{2R^2}\left(\frac{2\pi}{\lambda}\right)^4 \left(\frac{n^2-1}{n^2+2}\right)^2 \left(\frac{d}{2}\right)^6  $$
where $I_0$ is the original intensity, $\theta$ is the scattering angle, and $R$ is the range to the distance to the particle (Seinfeld and Pandis 2006).

The scatter cross section $\sigma_s$, after averaging over all scatter angles (Cox et al. 2002), is given by
$$ \sigma_s = \frac{2\pi^5 d^6}{3\lambda^4} \left(\frac{n^2-1}{n^2+2}\right)^2 .$$

You may also this relation with the polarisability $\alpha$ given by (Blue Sky 2015)
$$ I = I_0 \frac{8\pi^4 \alpha^2}{\lambda^4 R^2}(1+\cos^2\theta) .$$

Shorter wavelengths are scattered more as can be seen by the $\lambda^{-4}$ dependence so when we look at the sky on a sunny day, we can see more of the shorter wavelength blue light compared to red light. However, if there is heavy air pollution (possibly from bushfires), blue light is scattered away leaving the red light. 



## Atmospheric Absorption
According to Gao et al. (2009), eight gases in the atmosphere have measureable absorption effects for hyperspectral imagers in the wavelength range 400--3000 nm with spectral resolution 1--20 nm. These gases are: water vapour H$_2$O, carbon dioxide CO$_2$, ozone O$_3$, nitrous oxide N$_2$O, carbon monoxide CO, methane CH$_4$, oxygen O$_2$, and nitrogen dioxide NO$_2$. Water is by far the most absorbing and this is illustrated in Figure 3. 

![](../images/2020-04-27-basics_of_radiometric_correction_files/absorption.png "Figure 3: Atmospheric absorption by gas{% fn 4 %}."){:width="80%"}

### Transmittance
Attenuation coefficient $\tau$ and extinction coefficient $\alpha$
$$ \tau_\lambda(z) = \int_0^z \alpha(\lambda,z) dz $$

$$ T(z,\theta) = e^{-\tau_\lambda(z)/\cos\theta} $$
where $\tau_\lambda$ consists of the contributions from Rayleigh scattering, and absorption from water and ozone. 
$\cos\theta$ accounts for oblique view.


# System Effects


## Striping Artefacts
Noticable columns that are brighter than adjacent columns

![](../images/2020-04-27-basics_of_radiometric_correction_files/destriping.png "Figure 4: Hyperion band 128 - Coleambally, NSW{% fn 5 %}."){:width="80%"}


## Sensor Artefacts
spectral and spatial errors on detector array due to optical aging, degradations, aberrations, and misalignments in
pushbroom systems, especially in hyperspectral pushbroom systems where crosstrack and spectral pixels are continuously recorded at the same time using an area detector array.

Spectral error is smile, spatial error is keystone

![](../images/2020-04-27-basics_of_radiometric_correction_files/smile_keystone.png "Figure 5: Smile and keyston errors{% fn 6 %}."){:width="80%"}





### Destriping for Land
Procedure by Goetz et al. (2003)
great for terrestrial

pixel value (or digital number) $x_{ijk}$ for row $i$, column $j$, and band $k$
For each row $i$, we multiply by gain $a_{jk}$ and offset $b_{jk}$ such that each row's mean $\mu_{jk}$ and standard deviation $\sigma_{jk}$ is equal to the overall channel's mean $\mu_k$ and standard deviation $\sigma_k$.
$$ x_{ijk}' =  a_{jk}x_{ijk} + b_{jk} $$

$$ a_{jk} = \frac{\sigma_k}{\sigma_{jk}} $$

$$ b_{jk} = \mu_k - \frac{\sigma_k}{\sigma_{jk}} \mu_{jk} $$

This is applied Hyperion in Figure 4. 

#### Maximum Noise Fraction

like PCA based on noise covariance


### Destriping for Water
Water absorbs too much compared to land. Need to try something different.
Enter

Vertical Radiance Correction (VRadCor)

uses both spatial and spectral info
$$ R_i' (\lambda_k) = a_i(\lambda_k) R_i(\lambda_k) + b_i(\lambda_k)$$


### Correcting Smile and Keystone Errors
For smile, calibrate with absorption lines
assume spectral shift is linear across array

Integration, cubic spline method





# Radiometric Correction Models
$L_{obs}$ at sensor radiance, $L_a$ path radiance, $T_2$ sun-surface-sensor two way transmittance, $S$ spherical albedo, $E_s$ exoatmospheric solar irradiance, $\theta_s$ solar zenith angle, $\rho$ surface reflectance. 

$$ L_{obs} = L_a  + \frac{T_2 \rho}{1-\rho S}\frac{E_s\cos\theta_s}{\pi}  $$

$$ \rho = \frac{L_{obs}-L_a}{(L_{obs}-L_a)S+T_2 E_s\cos\theta_s/\pi} $$

## Absolute Correction

### MODTRAN
Moderate Resolution Atmospheric Radiance and Transmittance Model


### 6S
Second Simulation of the Satellite Signal in the Solar Spectrum


### FLAASH
Fast Line-of-sight Atmospheric Analysis of Spectral Hypercubes


### ELC
Empirical Line Calibration

linear regression against lab spectra, need to know materials in FOV.


### ATREM
Atmospheric Removal


### ATCOR
Atmospheric Correction

### HATCH

High-accuracy Atmospheric Correction for Hyperspectral data

### Comparison
No idea

## Relative Correction
same scene taken on different dates, normalised to reduce variations

# Python Code for a LOWTRAN7 Atmospheric Absorption Extinction Model

In [1]:
!pip install lowtran

     |████████████████████████████████| 849kB 2.8MB/s 
  Created wheel for lowtran: filename=lowtran-2.4.1-cp36-cp36m-linux_x86_64.whl size=614637 sha256=b0e0341b918068dd3edccb6b3b0c83d3da37e5fc43ab695ff413924b3d06571a
  Stored in directory: /root/.cache/pip/wheels/ab/11/99/c3fb68f5edced874ed59cfc332ccde23ef10ab84e97dc52b4e
Successfully built lowtran


In [2]:
#hide

#!/usr/bin/env python
"""
from https://github.com/space-physics/lowtran/blob/master/ScatterRadiance.py

Total Radiance = atmosphere rad. or boundary rad. + atm. scat. or boundary refl.
Lowtran outputs W cm^-2 ster^-1 micron^-1
we want photons cm^-2 s^-1 ster^-1 micron^-1
1 W cm^-2 = 10000 W m^-2
h = 6.62607004e-34 m^2 kg s^-1
I: irradiance
Np: numer of photons
Np = (Ilowtran*10000)*lambda_m/(h*c)
"""
from pathlib import Path
from matplotlib.pyplot import show
try:
    import seaborn as sns
    sns.set_context('talk')
except ImportError:
    pass
from argparse import ArgumentParser
import lowtran
from lowtran.plots import plotscatter


def scatterRadiance():
    p = ArgumentParser(description='Lowtran 7 interface')
    p.add_argument('-z', '--obsalt', help='altitude of observer [km]', type=float, default=0.)
    p.add_argument('-a', '--zenang', help='Observer zenith angle [deg] ', nargs='+', type=float, default=[0., 60, 80])
    p.add_argument('-s', '--short', help='shortest wavelength nm ', type=float, default=400)
    p.add_argument('-l', '--long', help='longest wavelength nm ', type=float, default=700)
    p.add_argument('-step', help='wavelength step size cm^-1', type=float, default=20)
    p.add_argument('-o', '--outfn', help='NetCDF4 file to write')
    p.add_argument('--model', help='0-6, see Card1 "model" reference. 5=subarctic winter', type=int, default=5)

    P = p.parse_args()

    c1 = {'model': P.model,
          'h1': P.obsalt,  # of observer
          'angle': P.zenang,  # of observer
          'wlshort': P.short,
          'wllong': P.long,
          'wlstep': P.step,
          }

    TR = lowtran.scatter(c1)

    if P.outfn:
        outfn = Path(P.outfn).expanduser()
        print('writing', outfn)
        TR.to_netcdf(outfn)

    plotscatter(TR, c1)

    show()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# from https://github.com/space-physics/lowtran/blob/master/ScatterRadiance.py
scatterRadiance()

usage: ipykernel_launcher.py [-h] [-z OBSALT] [-a ZENANG [ZENANG ...]]
                             [-s SHORT] [-l LONG] [-step STEP] [-o OUTFN]
                             [--model MODEL]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-88b9aac2-df0a-451e-b188-dc8e5cc45a40.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# References


{{ 'Pu, R. (2017). Hyperspectral remote sensing: Fundamentals and practices. CRC Press.' | fndetail: 1 }}

{{ 'Modified from Miller, S. W., and Vermote, E., NPOESS Visible/Infrared Imager/Radiometer: Algorithm Theoretical Basis Document, Version 5, Raytheon, Lanham, Maryland, 2002.' | fndetail: 2 }}

{{ 'From [wikipedia article on Mie scattering](https://astronomy.stackexchange.com/questions/10771/sunsets-mars-earth) with [stackexchange annotations](https://astronomy.stackexchange.com/questions/10771/sunsets-mars-earth)' | fndetail: 3 }}

{{ 'From Jensen, J. R., Remote Sensing of the Environment: An Earth Resource Perspective, 2nd ed., Pearson Education, Inc., New York, 2007.' | fndetail: 4 }}


{{ '©2003 IEEE. From Goetz, A. F. H., Kindel, B. C., Ferri, M., and Qu, Z., IEEE Transactions on Geoscience and Remote Sensing, 41, 1215−1221, 2003' | fndetail: 5 }}

{{ 'Yokoya, N. et al., Applied Optics, 49(24), 4568–4575, 2010.' | fndetail: 6 }}

{{ '' | fndetail: 7 }}


